In [2]:
import pandas as pd
import numpy as np


In [10]:
df = pd.read_csv("traffic/output_files/2_lane_no_lights_20_cars_button_slowdown_AGENT_DATA.csv")
df.head(20)

,Step,AgentID,X,Y,velocity,current_lane
0,1,0,0.4,50.0,[0.4272 0. ],0
1,1,1,0.0,50.0,[0.5 0. ],0
2,1,2,0.0,50.0,[0.5 0. ],0
3,1,3,0.0,50.0,[0.5 0. ],0
4,1,4,0.0,50.0,[0.5 0. ],0
5,1,5,0.0,50.0,[0.5 0. ],0
6,1,6,0.0,50.0,[0.5 0. ],0
7,1,7,0.0,50.0,[0.5 0. ],0
8,1,8,0.0,50.0,[0.5 0. ],0
9,1,9,0.0,50.0,[0.5 0. ],0
